In [16]:
import time

from rq import Queue
from redis import Redis
from rq.job import Job

from tasks import print_name, sample

In [129]:
# Connect to the Redis server
redis_conn = Redis(host='localhost', port=6379)

# Create a queue ("low" is the name of the queue. If not defined, default queue is used)
queue1 = Queue('low',connection=redis_conn, default_timeout=15) # default_timeout sets the time_out for every task in the queue 
queue2 = Queue(connection=redis_conn, default_timeout=15)

In [130]:
# Enqueue tasks with arguments
job1 = queue1.enqueue(sample, result_ttl=3600, job_id='sample_1')
job2 = queue1.enqueue(print_name, "Rahul")
job3 = queue1.enqueue(print_name, "Praveena", job_timeout=2)
# Demonstrate job IDs
print(f"Job1 ID: {job1.id}")
print(f"Job2 ID: {job2.id}")
print(f"Job3 ID: {job3.id}")

print(job1.origin)

Job1 ID: sample_1
Job2 ID: fa49f068-b89f-4fdb-b3ca-7c3e15239d46
Job3 ID: 30c0907e-690c-4f0d-9510-d93676b78db2
low


In [23]:
# Show queued jobs and their statuses
for job in queue1.jobs:
    print(f"Job ID: {job.id}, Queue: {queue1.name}, Status: {job.get_status()}, Result: {job.result}")

    # print(f"Job ID: {job.id}, Status: {job.get_status()}, Result: {job.result}")

Job ID: ecd7810e-a3c3-4880-a5d1-a16adbfd3ce1, Queue: low, Status: queued, Result: None


In [2]:
# job_ids_to_delete = ["88c3ac81-4795-4a86-835d-7452d47e3796" ] 

# # Delete the tasks from the queue
# queue2.delete(job_ids_to_delete)

In [115]:
# To enqueue multiple tasks in an single network call
jobs = queue1.enqueue_many(
  [
    Queue.prepare_data(print_name, ("Mary",) ),
    Queue.prepare_data(print_name, ("Prasad",) )
  ]
)

In [ ]:
# depends_on
from tasks import add,sample
job_add = queue1.enqueue(add, 5, 6)
queue1.enqueue(sample, depends_on = job_add) #task sample is completed only after add is successfully completed
job_add.result

## Scedule Jobs

In [122]:
# Create a queue
queue2 = Queue('new',connection=redis_conn, default_timeout=15)

In [123]:
from datetime import datetime, timedelta

# Schedule job to run at 2.15
time = datetime(year=2023, month=7, day=28, hour=15, minute=44, second=2)
job = queue2.enqueue_at(time, sample)



In [ ]:
# Schedule job to be run in 10 seconds
job = queue1.enqueue_in(timedelta(seconds=30), sample)

## Queue

In [124]:
# Enqueue tasks with arguments
job1 = queue1.enqueue(print_name, "Rahul")
job2 = queue1.enqueue(sample, result_ttl=3600, job_id='sample_1')
job3 = queue1.enqueue(print_name, "Praveena")

In [125]:
# Getting the number of jobs in the queue
# Note: Only queued jobs are counted, not including deferred ones
print(len(queue1))

# Retrieving jobs
queued_job_ids = queue1.job_ids # Gets a list of job IDs from the queue
queued_jobs = queue1.jobs # Gets a list of enqueued job instances

print("queued_job_ids:", queued_job_ids)
print("queued_jobs", queued_jobs)

1
queued_job_ids: ['ebdf98f1-89d1-4ad9-91e9-4004b9ac2cb9']
queued_jobs [Job('ebdf98f1-89d1-4ad9-91e9-4004b9ac2cb9', enqueued_at=datetime.datetime(2023, 7, 28, 10, 19, 34, 883483))]


In [ ]:
job = queue1.fetch_job('sample_1') # Returns job having ID "my_id"
print(job.result)

In [93]:
# Emptying a queue, this will delete all jobs in this queue
queue1.empty()

2

In [94]:
# Deleting a queue
queue1.delete(delete_jobs=True) # Passing in `True` will remove all jobs in the queue
# queue is now unusable. It can be recreated by enqueueing jobs to it.

In [110]:
from rq.command import send_stop_job_command

# This will raise an exception if job is invalid or not currently executing
send_stop_job_command(redis_conn, 'sample_1')

## Worker


In [83]:
from rq import Worker

# Provide the worker with the list of queues (str) to listen to.
w = Worker(['default'], connection=Redis())
w.work()

14:14:34 Worker rq:worker:5e25d4ce7f574770bc6c1a5de54a78d7 started with PID 6538, version 1.15.1
14:14:34 Subscribing to channel rq:pubsub:5e25d4ce7f574770bc6c1a5de54a78d7
14:14:34 *** Listening on default...
14:14:52 Worker 5e25d4ce7f574770bc6c1a5de54a78d7 [PID 6538]: warm shut down requested
14:14:52 Unsubscribing from channel rq:pubsub:5e25d4ce7f574770bc6c1a5de54a78d7


False

In [ ]:
# Provide the worker with the list of queues (str) to listen to.
w = Worker(['default'], connection=Redis(), name="abc")
w.work()

14:07:22 Worker rq:worker:abc started with PID 6538, version 1.15.1
14:07:22 Subscribing to channel rq:pubsub:abc
14:07:22 *** Listening on default...
14:07:27 Worker abc [PID 6538]: warm shut down requested
14:07:27 Unsubscribing from channel rq:pubsub:abc


False